# Get Started with Milvus

## Install Milvus C# SDK

In [1]:
#r "nuget: Milvus.Client, 2.2.2-preview.5"

Installed Packages Milvus.Client, 2.2.2-preview.5

In [2]:
using Milvus.Client;

## Initialize Milvus Client

In [3]:
var milvusClient = new MilvusClient("localhost", username: "username", password: "password");

## Cleanup (optional)

In [4]:
await milvusClient.GetCollection("movies").DropAsync()

## List collections

In [5]:
var collections = milvusClient.GetCollection("movies");

In [6]:
collections

Name,movies


## Create collection

### Define schema

In [7]:
var schema = new CollectionSchema
{
    Fields =
    {
        FieldSchema.Create<long>("movie_id", isPrimaryKey: true),
        FieldSchema.CreateVarchar("movie_name", maxLength: 200),
        FieldSchema.CreateFloatVector("movie_description", dimension: 2)
    },
    Description = "Test movie search",
    EnableDynamicFields = true
};

### Create collection

In [8]:
var collection = await milvusClient.CreateCollectionAsync(collectionName: "movies",schema: schema, shardsNum: 2);

In [9]:
await collection.DescribeAsync()

Milvus.Client.MilvusCollectionDescription Aliases [ ] CollectionName movies CollectionId 444190023189266532 ConsistencyLevel Session CreationTimestamp 1694450507115 Schema Milvus.Client.CollectionSchema Name movies Description Test movie search Fields index value 0 Milvus.Client.FieldSchema Name movie_id DataType Int64 IsPrimaryKey True AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension <null> State FieldCreated FieldId 100 1 Milvus.Client.FieldSchema Name movie_name DataType VarChar IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength 200 Dimension <null> State FieldCreated FieldId 101 2 Milvus.Client.FieldSchema Name movie_description DataType FloatVector IsPrimaryKey False AutoId False IsPartitionKey False Description IsDynamic False MaxLength <null> Dimension 2 State FieldCreated FieldId 102 EnableDynamicFields False ShardsNum 2 StartPositions (empty)

## Add data to collection

In [10]:
var movieIds = new [] {1L,2L,3L,4L,5L};
var movieNames = new [] {"The Lion King","Inception","Toy Story","Pulp Function","Shrek"};
var movieDescriptions = new ReadOnlyMemory<float>[] { 
        new [] {0.10022575f, -0.23998135f},
        new [] {0.10327095f, 0.2563685f},
        new [] {0.095857024f, -0.201278f},
        new [] {0.106827796f, 0.21676421f},
        new [] {0.09568083f, -0.21177962f}
    };

In [11]:
await collection.InsertAsync(new FieldData[]
{
    FieldData.Create("movie_id", movieIds),
    FieldData.Create("movie_name", movieNames),
    FieldData.CreateFloatVector("movie_description", movieDescriptions)
});

## Persist data

In [12]:
await collection.FlushAsync();

In [13]:
await collection.GetEntityCountAsync()

0

## Search for data

### Create Index

In [14]:
await collection.CreateIndexAsync(
    fieldName: "movie_description", 
    indexType: IndexType.Flat, 
    metricType: SimilarityMetricType.L2, 
    extraParams: new Dictionary<string,string> {["nlist"] = "1024"}, 
    indexName: "movie_idx");

### Load collection

In [15]:
await collection.LoadAsync();
await collection.WaitForCollectionLoadAsync();

### Define search parameters

In [16]:
var parameters = new SearchParameters
{
    OutputFields = { "movie_name" },
    ConsistencyLevel = ConsistencyLevel.Strong,
    ExtraParameters = { ["nprobe"] = "1024" }
};

### Search for data

Search for data using embedding vectors for the query "A movie that's fun for the whole family".

In [17]:
var results = await collection.SearchAsync(
    vectorFieldName: "movie_description",
    vectors: new ReadOnlyMemory<float>[] { new[] {0.12217915f, -0.034832448f } },
    SimilarityMetricType.L2,
    limit: 3,
    parameters);

In [18]:
results

Milvus.Client.SearchResults CollectionName movies FieldsData index value 0 Field: {FieldName: movie_name, DataType: VarChar, Data: 3, RowCount: 3} Data [ Toy Story, Shrek, The Lion King ] RowCount 3 FieldName movie_name FieldId 0 DataType VarChar IsDynamic False Ids Milvus.Client.MilvusIds LongIds [ 3, 5, 1 ] StringIds <null> NumQueries 1 Scores [ 0.028396975, 0.032012466, 0.042568024 ] Limit 3 Limits [ 3 ]

##